In [21]:
import xarray as xr 
import rioxarray as rio
import rasterio
import rasterio.plot
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import cartopy.crs as ccrs
import cartopy.feature as cfeature

#### Load & Read Files

In [22]:
file = "/home/kayatroyer/Repositories/netcdfpreprocessing/cordilleran/ciscyc.10km.grip.ex.1ka.nc"

grip_10km_1ka = xr.open_dataset(file)

### View NetCDF File

Dimensions, Coordinates, Data Variables, & Attributes

In [23]:
grip_10km_1ka

<xarray.Dataset> Size: 175MB
Dimensions:           (age: 120, y: 301, x: 151)
Coordinates:
    time              (age) object 960B ...
  * x                 (x) float64 1kB -2.5e+06 -2.49e+06 ... -1.01e+06 -1e+06
  * age               (age) float64 960B 119.0 118.0 117.0 ... 2.0 1.0 -0.0
  * y                 (y) float64 2kB 0.0 1e+04 2e+04 ... 2.99e+06 3e+06
Data variables:
    tempicethk_basal  (age, y, x) float32 22MB ...
    temppabase        (age, y, x) float32 22MB ...
    thk               (age, y, x) float32 22MB ...
    uvelbase          (age, y, x) float32 22MB ...
    uvelsurf          (age, y, x) float32 22MB ...
    vvelbase          (age, y, x) float32 22MB ...
    vvelsurf          (age, y, x) float32 22MB ...
    lon               (y, x) float32 182kB ...
    lat               (y, x) float32 182kB ...
    mapping           float64 8B ...
    pism_config       int8 1B ...
    topg              (age, y, x) float32 22MB ...
Attributes:
    Conventions:  CF-1.5
    command:      julien@altair 2020-01-12 19:15:15 UTC: /home/julien/git/wor...
    history:      julien@altair 2020-01-12 19:15:15 UTC: /home/julien/git/wor...
    proj4:        +units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95...
    source:       PISM stable v0.7.1-14-gb9fc6b0
    author:       Julien Seguinot
    title:        Cordilleran ice sheet glacial cycle simulations spatial dia...
    subtitle:     10km GRIP simulation
    institution:  Stockholm University, Sweden and ETH Zürich, Switzerland

#### Extracting Individual Variables

In [24]:
time = grip_10km_1ka['time']
thk = grip_10km_1ka['thk'] # ice thickness
bed = grip_10km_1ka['topg'] # bedrock surface elevation

uvel_base = grip_10km_1ka['uvelbase'] # x-component of the horizontal velocity of ice at the base of ice
vvel_base = grip_10km_1ka['vvelbase'] # y-component of the horizontal velocity of ice at the base of ice

uvel_surf = grip_10km_1ka['uvelsurf'] # x-component of the horizontal velocity of ice at ice surface
vvel_surf = grip_10km_1ka['vvelsurf'] # y-component of the horizontal velocity of ice at ice surface

Setting CRS

In [25]:
# ice thickness
thk = thk.rio.set_spatial_dims(x_dim='x', y_dim='y')
thk.rio.crs
thk.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# bedrock surface elevation
bed = bed.rio.set_spatial_dims(x_dim='x', y_dim='y')
bed.rio.crs
bed.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# x-component of the horizontal velocity of ice at the base of ice
uvel_base = uvel_base.rio.set_spatial_dims(x_dim='x', y_dim='y')
uvel_base.rio.crs
uvel_base.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# y-component of the horizontal velocity of ice at the base of ice
vvel_base = vvel_base.rio.set_spatial_dims(x_dim='x', y_dim='y')
vvel_base.rio.crs
vvel_base.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# x-component of the horizontal velocity of ice at ice surface
uvel_surf = uvel_surf.rio.set_spatial_dims(x_dim='x', y_dim='y')
uvel_surf.rio.crs
uvel_surf.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)

# y-component of the horizontal velocity of ice at ice surface
vvel_surf = vvel_surf.rio.set_spatial_dims(x_dim='x', y_dim='y')
vvel_surf.rio.crs
vvel_surf.rio.write_crs("+units=m +proj=lcc +lat_1=49 +lat_2=77 +lat_0=49 +lon_0=-95 +x_0=0 +y_0=0 +no_defs +a=6378137 +rf=298.257222101 +towgs84=0.000,0.000,0.000 +to_meter=1", inplace=True)


<xarray.DataArray 'vvelsurf' (age: 120, y: 301, x: 151)> Size: 22MB
[5454120 values with dtype=float32]
Coordinates:
    time     (age) object 960B ...
  * x        (x) float64 1kB -2.5e+06 -2.49e+06 -2.48e+06 ... -1.01e+06 -1e+06
  * age      (age) float64 960B 119.0 118.0 117.0 116.0 ... 3.0 2.0 1.0 -0.0
  * y        (y) float64 2kB 0.0 1e+04 2e+04 3e+04 ... 2.98e+06 2.99e+06 3e+06
    mapping  int64 8B 0
Attributes:
    units:        m year-1
    valid_range:  [-1000000.  1000000.]
    long_name:    y-component of the horizontal velocity of ice at ice surface
    pism_intent:  diagnostic

## Save to GeoTIFF File

In [26]:
# ice thickness
thk.rio.to_raster(r"ice_thickness.tif")

# bedrock surface elevation
bed.rio.to_raster(r"bed_topo.tif")

# x-component of the horizontal velocity of ice at the base of ice
uvel_base.rio.to_raster(r"x_basal_velocity.tif")

# y-component of the horizontal velocity of ice at the base of ice
vvel_base.rio.to_raster(r"y_basal_velocity.tif")

# x-component of the horizontal velocity of ice at ice surface
uvel_surf.rio.to_raster(r"x_surf_velocity.tif")

# y-component of the horizontal velocity of ice at ice surface
vvel_surf.rio.to_raster(r"y_surf_velocity.tif")

In [27]:
print(uvel_base.rio.crs)


EPSG:3978


In [28]:
tif = "/home/kayatroyer/Repositories/netcdfpreprocessing/cordilleran/ice_thickness.tif"

In [29]:
try:
    with rasterio.open("/home/kayatroyer/Repositories/netcdfpreprocessing/cordilleran/ice_thickness.tif") as src:
        image_array = src.read()  # Read all bands as a NumPy array
        print(f"Image array shape: {image_array.shape}")
        print(f"Coordinate Reference System (CRS): {src.crs}")
        print(f"Transform: {src.transform}")
except FileNotFoundError:
    print("Error: The specified GeoTIFF file was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Image array shape: (120, 301, 151)
Coordinate Reference System (CRS): EPSG:3978
Transform: | 10000.00, 0.00,-2505000.00|
| 0.00, 10000.00,-5000.00|
| 0.00, 0.00, 1.00|


### Visualize Data

In [30]:
# data_crs = ccrs.epsg(3978)
# fig,ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': data_crs})

# bounds = bed.rio.bounds()
# ax.set_extent([bounds[0], bounds[2], bounds[1], bounds[3]], crs=data_crs)

# ax.coastlines(resolution='10m')
# ax.add_feature(cfeature.BORDERS)
# ax.add_feature(cfeature.LAND, facecolor='lightgray')
# ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
# ax.gridlines(draw_labels=True)

# first_frame = bed.isel(age=0).values
# img = ax.imshow(first_frame, cmap='viridis', origin='lower',
#                 extent=bounds, transform=data_crs)
# cbar = fig.colorbar(img, ax=ax, orientation='vertical', fraction=0.046, pad=0.04)
# cbar.set_label('Ground ELevation (meters)') 

# def update(i):
#     data = bed.isel(age=i)
#     img.set_data(data)
#     # title.set_text(f"Time: {data['age'].values}")
#     return [img]

# ani = animation.FuncAnimation(fig, update, frames=bed.sizes["age"], interval = 200, blit = False)
# # HTML(ani.to_jshtml())
# ani.save('bed_grip.gif', writer='Pillow', fps=20)